# Tutorial: Generative QA with Retrieval Augmented Generation

In this tutorial, you'll learn how to run generative question answering by connecting a retriever to a generative LLM. You'll also learn how to use prompts with a generative model to tune your answers. The system should also generate a response like "Unanswerable" if no evidence is found.

You can plug-and-play this tutorial with most models on the HuggingFace model hub and also OpenAI LLMs. Some supported models include:
 - FLAN UL2-20B
 - FLAN T5 
 - Open AI ChatGPT (gpt-3.5-turbo)
 - InstructGPT(text-davinci-003)
 - lots more..

# Installing PrimeQA
First, we need to include the required modules.

In [ ]:
%%bash

pip install --upgrade pip
pip install primeqa

## Initialize the Retriever

### Pre-process your document collection here to be ready to be stored in your Neural Search Index.
In this step we download a publicly available .csv file from a Google Drive location and save it as .tsv.

In [1]:
# save your input document as a .tsv
import pandas as pd
url='https://drive.google.com/file/d/1LULJRPgN_hfuI2kG-wH4FUwXCCdDh9zh/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df.to_csv('input.tsv', sep='\t')

### Initialize the model. In PrimeQA we use the SearchableCorpus class for searching through your corpus.

For DPR, you need to point to a question and context encoder models available via the HuggingFace model hub.

In [2]:
from primeqa.util import SearchableCorpus
retriever = SearchableCorpus(context_encoder_name_or_path="PrimeQA/XOR-TyDi_monolingual_DPR_ctx_encoder",
                             query_encoder_name_or_path="PrimeQA/XOR-TyDi_monolingual_DPR_qry_encoder",
                             batch_size=64, top_k=10)

{"time":"2023-06-17 22:53:50,387", "name": "faiss.loader", "level": "INFO", "message": "Loading faiss with AVX2 support."}
{"time":"2023-06-17 22:53:50,396", "name": "faiss.loader", "level": "INFO", "message": "Successfully loaded faiss with AVX2 support."}


### Add your documents into the searchable corpus through PrimeQA's built-in pre-processor.

PrimeQA has a built-in class called DocumentCollection which pre-processes input.tsv to match the following format as needed by DPR:

`id \t text \t title_of_document`

Note: since DPR is based on an encoder language model the typical sequence length is 512 max sub-word tokens. So please make sure your documents are split into text length of ~220 words.

In [3]:
retriever.add_documents("input.tsv")

{"time":"2023-06-17 22:53:58,797", "name": "primeqa.ir.dense.dpr_top.dpr.index_simple_corpus", "level": "INFO", "message": "wrote passages_1_of_1.json.gz.records in 0 seconds"}
{"time":"2023-06-17 22:53:58,798", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "building index, reading data from dpr_index_dir/passages_1_of_1.json.gz.records, writing to dpr_index_dir/index_1_of_1.faiss"}
{"time":"2023-06-17 22:53:58,798", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "processed 0 passages"}
{"time":"2023-06-17 22:53:58,801", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "calling index.add with 76 vectors"}
{"time":"2023-06-17 22:53:58,802", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "processed 76 passages"}
{"time":"2023-06-17 22:53:58,802", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "finished building index, writing 

# Initialize the Reader 

In this step you can use a generative LLM which can be prompted. This reader can be any of the generative models available in the HuggingFace model hub or OpenAI models.

In [4]:
from primeqa.components.reader import GenerativeReader

reader = GenerativeReader(model_type='HuggingFace', model_name='google/flan-t5-small')
# setup an OpenAI generative reader : we support gpt-3.5-turbo and text-davinci-003
# reader = GenerativeReader(model_type='OpenAI', model_name='text-davinci-003', api_key='')

# Setup the RAG pipeline

Attach a retriever to a generative LLM. You can then prompt it to answer questions.

In [5]:
from primeqa.pipelines import RAG
pipeline = RAG(retriever, reader)

# Start asking questions

We "run" the pipeline we just created and also attach a prompt.

In [6]:
questions = ['When was Idaho split in two?' , 'Who was Danny Nozel']
prompt_prefix = "Answer the following question after looking at the text."

answers = pipeline.run(questions, prefix=prompt_prefix)

In [7]:
import pandas as pd
from IPython.display import display, HTML

output = pd.DataFrame.from_records(answers)
display(HTML(output.to_html()))

,question,answer,passages
0,When was Idaho split in two?,1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03-29 ; 1898-2000-03,"[Passage: ""American Revolutionary War""..., Passage: ""American Civil War""..., Passage: ""American Revolutionary War""...]"
1,Who was Danny Nozel,a sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war sailor of the american civil war s,"[Passage: Alaska..., Passage: ""Alexander Graham Bell""..., Passage: ""Alkali metal""...]"
